In [1]:
import pandas as pd
from glob import glob
from datetime import datetime
import random
import calendar
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind
import numpy as np

In [2]:
def cal_map_cat(x,dict_,i):
    if x[i] in dict_:
        return dict_[x[i]]
    else:
        return x[i]

class kv_pair(dict): 
    def __init__(self): 
        self = dict() 
    def add(self, key, value): 
        self[key] = value 
def cal_diff_seconds(x):
    return (x['trip_stop_time']-x['trip_start_time']).total_seconds()

def swap_columns(x,col1,col2,col3):
    if any(char.isalpha() for char in x[col3]):
        return x[col1]
    else:
        return x[col2]


In [3]:

dict_dfs=kv_pair()
for j in range(len(glob('../data-toronto/*.csv'))):
    i=glob('../data-toronto/*.csv')[j]
    df_temp=pd.read_csv(i,encoding='cp1252')
    if 'ï»¿Trip Id' in df_temp.columns:
        df_temp=df_temp.rename(columns={'ï»¿Trip Id':'Trip Id'})
    if i=='../data-toronto\\2020-10.csv':
        # for k in [['Start Station Id','Start Time']]:
            # df_temp=df_temp.apply(pat_prof, args=(i,),axis=1)
        df_temp['Start Time_copy']=df_temp['Start Time'].copy()
        df_temp['End Time_copy']=df_temp['End Time'].copy()
        df_temp['Start Time']=df_temp.apply(swap_columns,args=('Start Station Id','Start Time','Start Time_copy',),axis=1)
        df_temp['Start Station Id']=df_temp.apply(swap_columns,args=('Start Time_copy','Start Station Id','Start Time_copy',),axis=1)
        df_temp['End Time']=df_temp.apply(swap_columns,args=('End Station Id','End Time','End Time_copy',),axis=1)
        df_temp['End Station Id']=df_temp.apply(swap_columns,args=('End Time_copy','End Station Id','End Time_copy',),axis=1)

        
        df_temp=df_temp.drop(columns=['Start Time_copy'])
    if 'Start Station Id' in df_temp.columns:
        df_temp=df_temp.rename(columns={
            
            'Trip Id':'trip_id','Trip  Duration':'trip_duration_seconds','Start Station Id':'from_station_id','Start Time':'trip_start_time',
                               'Start Station Name':'from_station_nam','End Station Id':'to_station_id','End Time':'trip_stop_time',
                            'End Station Name':'to_station_name','Bike Id':'bike_id','User Type':'user_type'
                                       })

    df_temp['trip_start_time']=pd.to_datetime(df_temp['trip_start_time'])
    df_temp['trip_stop_time']=pd.to_datetime(df_temp['trip_stop_time'])
    if 'Duration' not in df_temp.columns:
        df_temp['Duration']=df_temp.apply(cal_diff_seconds,axis=1)  
    dict_dfs.add(j,df_temp.copy())
df=pd.DataFrame()
for i in dict_dfs:
    df=pd.concat([df,dict_dfs[i]])
df=df.rename(columns={
    'trip_id':'ride_id','from_station_id':'start_station_id','trip_start_time':'started_at',
                       'from_station_nam':'start_station_name','to_station_id':'end_station_id','trip_stop_time':'ended_at',
                        'to_station_name':'end_station_name','bike_id':'Bike number'
                               })

In [4]:
df2 = df.copy()

In [5]:
df

,ride_id,trip_duration_seconds,start_station_id,started_at,start_station_name,end_station_id,ended_at,end_station_name,Bike number,member_casual,Duration,End Time_copy,from_station_name
0,4581278,1547.0,7021,2019-01-01 00:08:00,Bay St / Albert St,7233.0,2019-01-01 00:33:00,King / Cowan Ave - SMART,1296,Annual Member,1500.0,NaN,NaN
1,4581279,1112.0,7160,2019-01-01 00:10:00,King St W / Tecumseth St,7051.0,2019-01-01 00:29:00,Wellesley St E / Yonge St (Green P),2947,Annual Member,1140.0,NaN,NaN
2,4581280,589.0,7055,2019-01-01 00:15:00,Jarvis St / Carlton St,7013.0,2019-01-01 00:25:00,Scott St / The Esplanade,2293,Annual Member,600.0,NaN,NaN
3,4581281,259.0,7012,2019-01-01 00:16:00,Elizabeth St / Edward St (Bus Terminal),7235.0,2019-01-01 00:20:00,Bay St / College St (West Side) - SMART,283,Annual Member,240.0,NaN,NaN
4,4581282,281.0,7041,2019-01-01 00:19:00,Edward St / Yonge St,7257.0,2019-01-01 00:24:00,Dundas St W / St. Patrick St,1799,Annual Member,300.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
363485,4581273,379.0,7088,2018-12-31 23:43:00,NaN,7091,2018-12-31 23:49:00,Donlands Station,NaN,Annual Member,360.0,NaN,Danforth Ave / Coxwell Ave
363486,4581274,306.0,7030,2018-12-31 23:45:00,NaN,7031,2018-12-31 23:50:00,Jarvis St / Isabella St,NaN,Annual Member,300.0,NaN,Bay St / Wellesley St W
363487,4581275,340.0,7020,2018-12-31 23:49:00,NaN,7000,2018-12-31 23:55:00,Fort York Blvd / Capreol Ct,NaN,Annual Member,360.0,NaN,Phoebe St / Spadina Ave
363488,4581276,1466.0,7014,2018-12-31 23:52:00,NaN,7269,2019-01-01 00:17:00,Toronto Eaton Centre (Yonge St),NaN,Annual Member,1500.0,NaN,Sherbourne St / Carlton St (Allan Gardens)


In [7]:
df2.to_csv('df.csv',index=False)

In [5]:

df=df[~df['user_type'].isnull()]
dict_cas={'Annual Member':'member','Casual Member':'casual'}
day = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
df["weekday"] = df['started_at'].apply(lambda x: pd.Timestamp(x).dayofweek)
df["weekday"] = df["weekday"].apply(lambda x: day[x])
df.head()
df['started_at']=pd.to_datetime(df['started_at'])
df['ended_at']=pd.to_datetime(df['ended_at'])
df['month']=df['started_at'].apply(lambda x: x.month)
df['year']=df['started_at'].apply(lambda x: x.year)

In [9]:
df_final=df[(~df['end_station_id'].isnull())&(~df['start_station_id'].isnull())]
len(df_final)

10844989

In [10]:
def fix_id(x,i):
    try:
        a=str(float(x[i]))
        return a
    except:
        a=x[i]
        return a
df_final['end_station_id']=df_final.apply(fix_id,args=('end_station_id',),axis=1)
df_final['start_station_id']=df_final.apply(fix_id,args=('start_station_id',),axis=1)


C:\Users\akbarimm\AppData\Local\Temp\ipykernel_22712\921780819.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['end_station_id']=df_final.apply(fix_id,args=('end_station_id',),axis=1)
C:\Users\akbarimm\AppData\Local\Temp\ipykernel_22712\921780819.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['start_station_id']=df_final.apply(fix_id,args=('start_station_id',),axis=1)


In [11]:
loc = pd.read_csv("../station_information.csv")
loc['station_id']=loc.apply(fix_id,args=('station_id',),axis=1)
loc=loc[(loc['station_id'].isin(df_final['end_station_id'].unique()))&(loc['station_id'].isin(df_final['end_station_id'].unique()))]
def fix_id(x,i):
    try:
        a=str(float(x[i]))
        return a
    except:
        a=x[i]
        return a


In [12]:
loc['start_lat']=loc['lat'].copy()
loc['end_lat']=loc['lat'].copy()
loc['start_lng']=loc['lon'].copy()
loc['end_lng']=loc['lon'].copy()
loc['start_station_name']=loc['name'].copy()
loc['end_station_name']=loc['name'].copy()
loc['start_station_id']=loc['station_id'].copy()
loc['end_station_id']=loc['station_id'].copy()
loc['start_station_capacity']=loc['capacity'].copy()
loc['end_station_capacity']=loc['capacity'].copy()
df_final2=pd.merge(df_final[[i for i in df_final.columns if i not in ['start_lng_rounded','start_lat_rounded','start_lng','start_lat']]],loc[['start_station_id','start_lat','start_lng','start_station_capacity']],on=['start_station_id'],how='outer')
len(df_final2)
df_final2=pd.merge(df_final2[[i for i in df_final2.columns if i not in ['end_lng_rounded','end_lat_rounded','end_lng','end_lat']]],loc[['end_station_id','end_lat','end_lng','end_station_capacity']],on=['end_station_id'],how='outer')
len(df_final2)

10844989

In [13]:
####We focused on data from 2018-2021 with available start and end stations and their longtitute and latitude. which resulted in 9196635 records. 
df_final2 = df_final2[(~df_final2['start_lat'].isnull())&(~df_final2['start_lng'].isnull())&(~df_final2['end_lat'].isnull())&(~df_final2['end_lng'].isnull())]


In [14]:
len(df_final2)

9196635

In [15]:
df_final2['started_at']=pd.to_datetime(df_final2['started_at'])
df_final2['ended_at']=pd.to_datetime(df_final2['ended_at'])
df_final2['started_hour'] = df_final2['started_at'].dt.hour
dict_period={0:'Early Morning',1:'Early Morning',2:'Early Morning',3:'Early Morning',4:'Early Morning',5:'Early Morning',
         6:'Morning',7:'Morning',8:'Morning',9:'Morning',10:'Morning',11:'Morning',
         12:'Afternoon',13:'Afternoon',14:'Afternoon',15:'Afternoon',16:'Afternoon',17:'Afternoon',
         18:'Night',19:'Night',20:'Night',21:'Night',22:'Night',23:'Night'}
df_final2['period']=df_final2['started_hour'].copy()
df_final2['period']=df_final2.apply(cal_map_cat, args=(dict_period,'period',), axis=1)
# df_final2.to_csv('Toronto_data.csv',index=False)
# loc.to_csv('Toronto_station_data.csv',index=False)